<a href="https://colab.research.google.com/github/wenchun0731/city-hunter/blob/main/%E7%B4%80%E9%8C%84%E8%BB%8A%E5%AD%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#掛載雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#下載ultralytics開源庫
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.9/871.9 kB 18.6 MB/s eta 0:00:00


In [3]:
#下載YOLOv8並檢查模型配置
from ultralytics import YOLO
import os
from IPython.display import display,Image
from IPython import display
display.clear_output()#清除当前输出单元格的所有输出内容
!yolo model=checks#检查 YOLO 模型的状态或配置信息。

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 760, in entrypoint
    check_dict_alignment(full_args_dict, {a: ""})
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 440, in check_dict_alignment
    raise SyntaxError(string + CLI_HELP_MSG) from e
SyntaxError: 'YOLO' is not a valid YOLO argument. 

    Arguments received: ['yolo', 'model=checks#检查', 'YOLO', '模型的状态或配置信息。']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'segment', 'classify', 'pose', 'obb', 'detect'}
                MODE (required) is one of {'train', 'val', 'benchmark', 'track', 'export', 'predict'}
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ul

In [ ]:
#Deep sort 原程式
#DeepSort(使用自己的訓練檔)
!pip install ultralytics
!pip install deep-sort-realtime
import numpy as np
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab.patches import cv2_imshow

#輸入
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')
cap = cv2.VideoCapture('/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/test_video4.mp4')  #輸入影片
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fNUMS = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/test-video0905.mp4", fourcc, fps, size)#輸出位置


#nms_max_overlap非極大值抑制（NMS）過程中允許的最大重疊比例
#nn_budget控制最近鄰搜尋中使用的樣本數量的上限
#max_cosine_distance 控制追蹤器在匹配檢測框與追蹤框時，外觀特徵向量之間的最大允許餘弦距離，一般設置為0到1之間
tracker = DeepSort(
    max_age=60,  # 增加此值以保持更長時間的追蹤
    max_cosine_distance=0.4,  # 減少此值以提高特徵匹配的精確度一般設置為0到1之間
    nn_budget=100  # 調整最近鄰搜尋的樣本數量
)

# 定義顏色字典
colors = {
    'ambulance':(0,255,0),   # 綠色
    'bus': (0,130,255),     # 青色
    'car': (0,0,255),      # 藍色
    'fire truck': (255,0,0),  # 紅色
    'licence': (60,0,128),   # 紫色
    'person': (255,255,0),   # 黃色
    'police': (0,0,0),     # 黑色
    'traffic light': (255,0,255) # 粉紅色
}

def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        # 在這裡將座標加入標籤內容
        label_with_coords = f"{label} ({p1[0]},{p1[1]})-({p2[0]},{p2[1]})"
        w, h = cv2.getTextSize(label_with_coords, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image,
                    label_with_coords, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2/3, txt_color, thickness=3, lineType=cv2.LINE_AA)


while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model(frame, conf=0.5)#置信值
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                class_id = int(output[5])

                # 根據模型的類別對應，將class_id對應到具體的類別名稱
                if class_id == 0:
                    label = 'ambulance'
                elif class_id == 1:
                    label = 'bus'
                elif class_id == 2:
                    label = 'car'
                elif class_id == 3:
                    label = 'fire truck'
                elif class_id == 4:
                    label = 'licence'
                elif class_id == 5:
                    label = 'person'
                elif class_id == 6:
                    label = 'police'
                elif class_id == 7:
                    label = 'traffic light'

                color = colors.get(label, (128, 128, 128))  # 默認顏色

                detections.append(([x1, y1, int(x2-x1), int(y2-y1)], output[4], label))

            tracks = tracker.update_tracks(detections, frame=frame)

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()

                # 獲取對應的顏色
                label = track.det_class
                color = colors.get(label, (128, 128, 128))  # 默認顏色
                box_label(frame, bbox, '#' + str(int(track_id)) + label, color)

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()


0: 384x640 2 cars, 40.5ms
Speed: 1.8ms preprocess, 40.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 30.3ms
Speed: 3.8ms preprocess, 30.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 30.3ms
Speed: 2.3ms preprocess, 30.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 30.4ms
Speed: 2.1ms preprocess, 30.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 25.7ms
Speed: 2.0ms preprocess, 25.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 47.3ms
Speed: 2.5ms preprocess, 47.3ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 25.4ms
Speed: 2.0ms preprocess, 25.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 25.3ms
Speed: 2.1ms preprocess, 25.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x6

In [ ]:
#不能用
!pip install ultralytics
!pip install deep-sort-realtime
import numpy as np
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab.patches import cv2_imshow

# 載入模型
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')
cap = cv2.VideoCapture('/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/test_video4.mp4')  # 輸入影片
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/test-video0909.mp4", fourcc, fps, size)  # 輸出位置

# 初始化 DeepSort 追蹤器
tracker = DeepSort(
    max_age=60,
    max_cosine_distance=0.4,
    nn_budget=100
)

# 定義顏色字典
colors = {
    'ambulance': (0, 255, 0),
    'bus': (0, 130, 255),
    'car': (0, 0, 255),
    'fire truck': (255, 0, 0),
    'licence': (60, 0, 128),
    'person': (255, 255, 0),
    'police': (0, 0, 0),
    'traffic light': (255, 0, 255)
}

# 偵測白線並畫出黃色線條
def detect_white_line(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 轉為灰階
    edges = cv2.Canny(gray, 50, 150)  # 邊緣檢測
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=50)

    white_line = None
    frame_height, frame_width = frame.shape[:2]
    middle_area_start = int(frame_height * 0.4)  # 定義畫面中間的範圍
    middle_area_end = int(frame_height * 0.6)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(y1 - y2) < 10 and middle_area_start < y1 < middle_area_end:  # 確保線條幾乎水平並位於中間區域
                white_line = (x1, y1, x2, y2)
                cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 255), 5)  # 畫出黃色線條（黃色）

    return white_line

# 偵測紅綠燈顏色
def get_traffic_light_color(detections):
    for det in detections:
        label = det[2]
        if label == 'traffic light':
            return label  # 假設模型可以辨識紅燈
    return None

# 判斷闖紅燈
def check_red_light_violation(white_line, tracks, traffic_light_color, frame):
    if white_line is None or traffic_light_color != 'red':
        return

    for track in tracks:
        if track.det_class in ['car', 'bus', 'ambulance', 'fire truck']:  # 只考慮車輛類別
            bbox = track.to_ltrb()
            if bbox[1] > white_line[1]:  # 車輛超過白線
                print(f"Car {track.track_id} ran a red light!")
                box_label(frame, bbox, f'Violation #{track.track_id}', (0, 0, 255))

# 畫框標籤
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        label_with_coords = f"{label} ({p1[0]},{p1[1]})-({p2[0]},{p2[1]})"
        w, h = cv2.getTextSize(label_with_coords, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image, label_with_coords, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2 / 3, txt_color, thickness=3, lineType=cv2.LINE_AA)

# 主程式迴圈
while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model(frame, conf=0.5)  # 偵測物體
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []
        white_line = detect_white_line(frame)  # 偵測白線

        if outputs is not None:
            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                class_id = int(output[5])

                label = ['ambulance', 'bus', 'car', 'fire truck', 'licence', 'person', 'police', 'traffic light'][class_id]
                color = colors.get(label, (128, 128, 128))
                detections.append(([x1, y1, int(x2 - x1), int(y2 - y1)], output[4], label))

            traffic_light_color = get_traffic_light_color(detections)  # 偵測紅綠燈顏色
            tracks = tracker.update_tracks(detections, frame=frame)  # 更新追蹤資訊

            check_red_light_violation(white_line, tracks, traffic_light_color, frame)  # 檢查是否闖紅燈

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()

                label = track.det_class
                color = colors.get(label, (128, 128, 128))
                box_label(frame, bbox, '#' + str(int(track_id)) + label, color)

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 74.6 MB/s eta 0:00:00

0: 384x640 2 cars, 82.6ms
Speed: 10.0ms preprocess, 82.6ms inference, 807.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 39.8ms
Speed: 1.9ms preprocess, 39.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 39.8ms
Speed: 2.2ms preprocess, 39.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 39.8ms
Speed: 2.4ms preprocess, 39.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 39.8ms
Speed: 1.9ms preprocess, 39.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 39.9ms
Speed: 2.0ms preprocess, 39.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 39.8ms
Speed: 1.9ms preprocess, 39.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 39.8ms
Speed: 3.1ms preprocess, 3

In [ ]:
# 標停止線
!pip install ultralytics
!pip install deep-sort-realtime

import numpy as np
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab.patches import cv2_imshow

# 載入模型
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')
cap = cv2.VideoCapture('/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/test_video4.mp4')  # 輸入影片
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/test-video0905.mp4", fourcc, fps, size)  # 輸出位置

# 初始化 DeepSort 追蹤器
tracker = DeepSort(
    max_age=60,
    max_cosine_distance=0.4,
    nn_budget=100
)

# 定義顏色字典
colors = {
    'ambulance': (0, 255, 0),
    'bus': (0, 130, 255),
    'car': (0, 0, 255),
    'fire truck': (255, 0, 0),
    'licence': (60, 0, 128),
    'person': (255, 255, 0),
    'police': (0, 0, 0),
    'traffic light': (255, 0, 255)
}

# 偵測紅綠燈顏色
def get_traffic_light_color(detections):
    for det in detections:
        label = det[2]
        if label == 'traffic light':
            return label  # 假設模型可以辨識紅燈
    return None

# 判斷闖紅燈
def check_red_light_violation(tracks, traffic_light_color, frame):
    if traffic_light_color != 'red':
        return

    # 固定的停止線座標：x從630到900，y為330
    stop_line_y = 330
    for track in tracks:
        if track.det_class in ['car', 'bus', 'ambulance', 'fire truck']:  # 只考慮車輛類別
            bbox = track.to_ltrb()
            if bbox[1] > stop_line_y:  # 車輛超過停止線
                print(f"Car {track.track_id} ran a red light!")
                box_label(frame, bbox, f'Violation #{track.track_id}', (0, 0, 255))

# 畫框標籤
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        label_with_coords = f"{label} ({p1[0]},{p1[1]})-({p2[0]},{p2[1]})"
        w, h = cv2.getTextSize(label_with_coords, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image, label_with_coords, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2 / 3, txt_color, thickness=3, lineType=cv2.LINE_AA)

# 主程式迴圈
while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model(frame, conf=0.5)  # 偵測物體
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                class_id = int(output[5])

                label = ['ambulance', 'bus', 'car', 'fire truck', 'licence', 'person', 'police', 'traffic light'][class_id]
                color = colors.get(label, (128, 128, 128))
                detections.append(([x1, y1, int(x2 - x1), int(y2 - y1)], output[4], label))

            traffic_light_color = get_traffic_light_color(detections)  # 偵測紅綠燈顏色
            tracks = tracker.update_tracks(detections, frame=frame)  # 更新追蹤資訊

            check_red_light_violation(tracks, traffic_light_color, frame)  # 檢查是否闖紅燈

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()

                label = track.det_class
                color = colors.get(label, (128, 128, 128))
                box_label(frame, bbox, '#' + str(int(track_id)) + label, color)

        # 在畫面中繪製固定的黃色停止線（x: 630-1200，y: 430）
        cv2.line(frame, (630, 430), (1200, 430), (0, 255, 255), 5)  # 黃色停止線

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()



0: 384x640 2 cars, 40.8ms
Speed: 1.9ms preprocess, 40.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 39.8ms
Speed: 1.9ms preprocess, 39.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 39.4ms
Speed: 2.2ms preprocess, 39.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 27.8ms
Speed: 2.5ms preprocess, 27.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 27.8ms
Speed: 3.0ms preprocess, 27.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 27.8ms
Speed: 2.6ms preprocess, 27.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 24.8ms
Speed: 1.8ms preprocess, 24.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 24.3ms
Speed: 2.9ms preprocess, 24.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x6

In [4]:
# 紀錄經過停止線的車
!pip install ultralytics
!pip install deep-sort-realtime

import numpy as np
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# 載入模型
model = YOLO('/content/drive/MyDrive/Finetune/Licence_Finetune/train/train/weights/best.pt')
cap = cv2.VideoCapture('/content/drive/MyDrive/Finetune/Licence_Finetune/test_video/test_video4.mp4')  # 輸入影片
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter("/content/drive/MyDrive/test-video0911.mp4", fourcc, fps, size)  # 輸出位置

# 初始化 DeepSort 追蹤器
tracker = DeepSort(
    max_age=30,
    max_cosine_distance=0.4,
    nn_budget=100
)

# 定義顏色字典
colors = {
    'ambulance': (0, 255, 0),
    'bus': (0, 130, 255),
    'car': (0, 0, 255),
    'fire truck': (255, 0, 0),
    'licence': (60, 0, 128),
    'person': (255, 255, 0),
    'police': (0, 0, 0),
    'traffic light': (255, 0, 255)
}

# 用來存儲通過停止線的車輛資訊的列表
vehicles_passed_stop_line = []

# 畫框標籤
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(image, p1, p2, color, thickness=3, lineType=cv2.LINE_AA)
    if label:
        label_with_coords = f"{label} ({p1[0]},{p1[1]})-({p2[0]},{p2[1]})"
        w, h = cv2.getTextSize(label_with_coords, 0, fontScale=2 / 3, thickness=3)[0]
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)
        cv2.putText(image, label_with_coords, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, 2 / 3, txt_color, thickness=3, lineType=cv2.LINE_AA)

# 判斷車輛是否超過停止線
def check_vehicle_passing_stop_line(tracks, frame_num, frame):
    # 固定的停止線座標：x從630到900，y為430
    stop_line_y = 430
    for track in tracks:
        if track.det_class in ['car', 'bus', 'ambulance', 'fire truck']:  # 只考慮車輛類別
            bbox = track.to_ltrb()
            if bbox[1] > stop_line_y:  # 車輛超過停止線
                # 計算時間戳（影片當前幀數除以FPS）
                timestamp = frame_num / fps
                # 保存車輛ID、座標、時間戳
                vehicles_passed_stop_line.append({
                    'track_id': track.track_id,
                    'coordinates': bbox,
                    'timestamp': timestamp
                })
                box_label(frame, bbox, f'Passed #{track.track_id}', (0, 255, 255))

# 主程式迴圈
frame_num = 0
while cap.isOpened():
    success, frame = cap.read()

    if success:
        frame_num += 1
        results = model(frame, conf=0.5)  # 偵測物體
        outputs = results[0].boxes.data.cpu().numpy()

        detections = []

        if outputs is not None:
            for output in outputs:
                x1, y1, x2, y2 = list(map(int, output[:4]))
                class_id = int(output[5])

                label = ['ambulance', 'bus', 'car', 'fire truck', 'licence', 'person', 'police', 'traffic light'][class_id]
                color = colors.get(label, (128, 128, 128))
                detections.append(([x1, y1, int(x2 - x1), int(y2 - y1)], output[4], label))

            tracks = tracker.update_tracks(detections, frame=frame)  # 更新追蹤資訊

            check_vehicle_passing_stop_line(tracks, frame_num, frame)  # 檢查車輛是否通過停止線

            for track in tracks:
                if not track.is_confirmed():
                    continue
                track_id = track.track_id
                bbox = track.to_ltrb()

                label = track.det_class
                color = colors.get(label, (128, 128, 128))
                box_label(frame, bbox, '#' + str(int(track_id)) + label, color)

        # 在畫面中繪製固定的黃色停止線（x: 630-1200，y: 430）
        cv2.line(frame, (630, 430), (1200, 430), (0, 255, 255), 5)  # 黃色停止線

        cv2.putText(frame, "https://blog.csdn.net/zhaocj", (25, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        videoWriter.write(frame)

    else:
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()

# 打印所有通過停止線的車輛資訊
print("Vehicles that passed the stop line:")
for vehicle in vehicles_passed_stop_line:
    print(f"Vehicle ID: {vehicle['track_id']}, Coordinates: {vehicle['coordinates']}, Time: {vehicle['timestamp']:.2f} seconds")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 60.7 MB/s eta 0:00:00

0: 384x640 2 cars, 109.2ms
Speed: 5.6ms preprocess, 109.2ms inference, 703.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 40.0ms
Speed: 3.1ms preprocess, 40.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 39.9ms
Speed: 2.6ms preprocess, 39.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 39.9ms
Speed: 1.5ms preprocess, 39.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 39.9ms
Speed: 2.1ms preprocess, 39.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 39.9ms
Speed: 2.1ms preprocess, 39.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 32.9ms
Speed: 2.0ms preprocess, 32.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 32.9ms
Speed: 2.2ms preprocess, 